In [1]:
import os

In [2]:
%pwd

'd:\\MLops project\\project-dl-end-to-end-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLops project\\project-dl-end-to-end-main'

In [ ]:
https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow

In [ ]:
import dagshub
dagshub.init(repo_owner='surajkale99', repo_name='project-dl-end-to-end-main', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="surajkale99"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e4f2406488322cebc687e5c389841eb67f8beeec"

In [ ]:
export MLFLOW_TRACKING_URI=https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow

export MLFLOW_TRACKING_USERNAME=surajkale99

export MLFLOW_TRACKING_PASSWORD=e4f2406488322cebc687e5c389841eb67f8beeec

In [ ]:
set MLFLOW_TRACKING_URI=https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow
set MLFLOW_TRACKING_USERNAME=surajkale99
set MLFLOW_TRACKING_PASSWORD=e4f2406488322cebc687e5c389841eb67f8beeec


In [7]:
!pip install mlflow

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-

In [ ]:
import mlflow
import os


os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow"
# Set your DagsHub credentials (MLflow uses HTTP Basic Auth)
os.environ["MLFLOW_TRACKING_USERNAME"] = "surajkale99"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "e4f2406488322cebc687e5c389841eb67f8beeec"



# Set MLflow tracking URI
mlflow.set_tracking_uri("https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow")

# # Optional: Verify connection by starting a run
# with mlflow.start_run(run_name="test-connection"):
#     mlflow.log_param("test_param", 123)
#     mlflow.log_metric("test_metric", 0.99)

# print("Run logged to DagsHub MLflow successfully.")


In [9]:
import tensorflow as tf

In [10]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [11]:
# entity

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# configuration manager

In [15]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.comman import read_yaml,create_directories,save_json

In [6]:
import tensorflow as tf

In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [17]:
#components

In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [19]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [21]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.7973 - loss: 0.3401


2025/04/14 11:14:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/14 11:14:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/04/14 11:15:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


🏃 View run auspicious-chimp-108 at: https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow/#/experiments/0/runs/279e5cbe1f8b4b56ba9949ef95b8e70c
🧪 View experiment at: https://dagshub.com/surajkale99/project-dl-end-to-end-main.mlflow/#/experiments/0
